In [74]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
load_dotenv()


True

In [75]:
groq_api_keys=os.getenv("GROQ_APIKEY")

In [76]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\Gen AI Project\\7-Project-Ineuron'

In [77]:
%cd ..

c:\Users\Lenovo\Desktop\Gen AI Project


c:\Users\Lenovo\Desktop\Gen AI Project\7-Project-Ineuron\myenv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [78]:
from langchain.document_loaders import PyPDFLoader
loder=PyPDFLoader(r"C:\Users\Lenovo\Desktop\Gen AI Project\7-Project-Ineuron\Interview-Question-Creator\dataset\attention_all_you_need.pdf")
data = loder.load()
data

[Document(metadata={'source': 'C:\\Users\\Lenovo\\Desktop\\Gen AI Project\\7-Project-Ineuron\\Interview-Question-Creator\\dataset\\attention_all_you_need.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence a

In [79]:
len(data)

11

In [80]:
question_gen=''
for page in data:
    question_gen+=page.page_content

In [81]:
type(data)

list

In [82]:
type(question_gen)

str

In [83]:
question_gen

'Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantly\nless tim

In [84]:
text_split = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_doc = text_split.split_text(question_gen)
final_doc


['Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantly',
 'enti

In [85]:
len(final_doc)

41

In [86]:
from langchain.docstore.document import Document

documents_ques_gen = [Document(page_content=t) for t in final_doc]

In [87]:
documents_ques_gen

[Document(metadata={}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable 

In [88]:
llm =ChatGroq(groq_api_key=groq_api_keys,model_name='Llama3-8b-8192')

In [89]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [90]:
from langchain.prompts import PromptTemplate
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [91]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
from langchain.chains.summarize import load_summarize_chain

In [98]:
ques_gen_chain = load_summarize_chain(llm = llm, 
                                          chain_type = "refine",  
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)
ques = ques_gen_chain.run(documents_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism.

In [93]:
os.environ["GOOGLE_API_KEY"] = os.getenv('GEMINI_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [94]:
vdb=FAISS.from_documents(documents=documents_ques_gen ,embedding=embeddings)

In [97]:
ques

'Based on the revised context and additional references, I will refine the original questions to make them more specific and accurate. Here are the revised questions:\n\n1. What is the primary focus of the paper "Gradient flow in recurrent nets: the difficulty of learning long-term dependencies" [11]?\na) Investigating the challenges of training recurrent neural networks to learn long-term dependencies\nb) Developing a new algorithm for natural language processing\nc) Improving computer vision techniques for image recognition\nd) Enhancing the performance of structured attention networks\n\nOriginal answer: a) Investigating the challenges of training recurrent neural networks to learn long-term dependencies\n\n2. What is the primary focus of the paper "Outrageously large neural networks: The sparsely-gated mixture-of-experts layer" [27]?\na) Developing a new algorithm for natural language processing\nb) Improving computer vision techniques for image recognition\nc) Investigating the ch

In [99]:
ques_list=ques.split("\n")
ques_list

['Thank you for providing the refined questions and additional context. After reviewing the context, I recommend refining questions 11-15, 18, and 20 to better incorporate the additional information. Here are the refined questions:',
 '',
 '11. According to [16], what is the primary advantage of using self-attention mechanisms in the Transformer architecture, enabling the model to model long-range dependencies without recurrence or convolution?',
 '',
 "12. How does the Transformer architecture's reliance on self-attention mechanisms, as described in [16], improve its ability to handle sequences of varying lengths without difficulty, allowing the model to process input sequences of different lengths?",
 '',
 '13. What is the purpose of the scaled dot-product attention mechanism in the Transformer architecture, as described in [19], which computes attention weights for both input and output sequences?',
 '',
 "14. How does the Transformer architecture's reliance on self-attention mechan

In [100]:
from langchain.chains import RetrievalQA
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm, 
                                               chain_type="stuff", 
                                               retriever=vdb.as_retriever())

In [101]:
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  Thank you for providing the refined questions and additional context. After reviewing the context, I recommend refining questions 11-15, 18, and 20 to better incorporate the additional information. Here are the refined questions:
Answer:  I'm happy to help with refining questions! However, I need to clarify that there are no questions 11-15, 18, and 20 provided. Could you please provide the questions you'd like me to refine? I'll do my best to incorporate the additional context and provide refined questions for you.
--------------------------------------------------\n\n
Question:  
Answer:  I don't know the answer to your question.
--------------------------------------------------\n\n
Question:  11. According to [16], what is the primary advantage of using self-attention mechanisms in the Transformer architecture, enabling the model to model long-range dependencies without recurrence or convolution?
Answer:  The primary advantage of using self-attention mechanisms in the Tr

In [117]:
generate_questions_template = """
You are given the following text extracted from a PDF. Based on the text, generate a list of questions that cover all the key points, important information, concepts, and ideas. Focus on capturing the essence of the content.

Text: {pdf_text}

Generate questions:
"""

# Create a PromptTemplate for generating questions
question_prompt = PromptTemplate(
    input_variables=["pdf_text"],
    template=generate_questions_template
)

# Step 2: Refine Questions and Generate Answers Template
refine_questions_template = """
You are given the following list of questions. Refine and improve these questions, ensuring they are clear, concise, and accurately reflect the key information from the text.
Just give list f refined questions dnt give any there information
Questions:
{questions}

Generate refined questions:
"""

# Create a PromptTemplate for refining questions and generating answers
refine_prompt = PromptTemplate(
    input_variables=["questions"],
    template=refine_questions_template
)

In [104]:
from langchain.chains import LLMChain

In [118]:
generate_questions_chain = LLMChain(llm=llm, prompt=question_prompt)
refine_qa_chain = LLMChain(llm=llm, prompt=refine_prompt)


In [119]:
final_qa_output = ""
    
for chunk in documents_ques_gen:
    # Step 1: Generate questions from the current chunk
    questions_output = generate_questions_chain.run(chunk)
    
    # Step 2: Refine the questions and generate answers
    qa_output = refine_qa_chain.run(questions_output)
    
    # Append the results for each chunk
    final_qa_output += qa_output + "\n\n"

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j3tqz9d9fr7thvypm15njsmw` on : Limit 500000, Used 500344, Requested 452. Please try again in 2m17.6134s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [115]:
final_qa_output

'Here are the refined questions:\n\n1. What is the primary focus of the paper, and what issue does it aim to address in the field of sequence transduction models?\n\n2. What are the prevailing sequence transduction models currently employed, and what are their core components?\n\n3. What is the attention mechanism, and how is it utilized in the top-performing sequence transduction models?\n\n4. What is the Transformer architecture, and how does it differ from existing sequence transduction models in terms of its design and components?\n\n5. What are the key components of the Transformer architecture, and how do they interact to achieve effective sequence transduction?\n\n6. What are the advantages of using the Transformer over existing models, and how do these benefits impact its performance?\n\n7. How do the authors evaluate the effectiveness of the Transformer, and what are the outcomes of their experiments?\n\n8. What are the two machine translation tasks employed to assess the perf

In [116]:

ques_list=final_qa_output.split("\n")
ques_list

['Here are the refined questions:',
 '',
 '1. What is the primary focus of the paper, and what issue does it aim to address in the field of sequence transduction models?',
 '',
 '2. What are the prevailing sequence transduction models currently employed, and what are their core components?',
 '',
 '3. What is the attention mechanism, and how is it utilized in the top-performing sequence transduction models?',
 '',
 '4. What is the Transformer architecture, and how does it differ from existing sequence transduction models in terms of its design and components?',
 '',
 '5. What are the key components of the Transformer architecture, and how do they interact to achieve effective sequence transduction?',
 '',
 '6. What are the advantages of using the Transformer over existing models, and how do these benefits impact its performance?',
 '',
 '7. How do the authors evaluate the effectiveness of the Transformer, and what are the outcomes of their experiments?',
 '',
 '8. What are the two mach